In [1]:
import os

In [2]:
from keras.preprocessing import image

In [3]:
import cv2

In [4]:
categories = ['with_mask', 'without_mask']

In [5]:
data = []
for category in categories:
    path = os.path.join('train',category)
    
    label = categories.index(category)
    
    for file in os.listdir(path):
        img_path = os.path.join(path,file)
        img = cv2.imread(img_path)
        img = cv2.resize(img,(224,224))
        
        data.append([img,label])
    

In [6]:
import random

In [7]:
random.shuffle(data)

In [8]:
X = []
Y = []
for features,label in data:
    X.append(features)
    Y.append(label)

In [9]:
import numpy as np

In [10]:
X = np.array(X)
Y = np.array(Y)

In [11]:
X.shape

(1632, 224, 224, 3)

In [12]:
X.shape


(1632, 224, 224, 3)

In [13]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2)

In [14]:
X_train.shape

(1305, 224, 224, 3)

In [15]:
X_test.shape

(327, 224, 224, 3)

In [16]:
from keras.applications.vgg16 import VGG16

In [17]:
vgg = VGG16()

In [18]:
from keras import Sequential

In [19]:
model = Sequential()

In [20]:
for layer in vgg.layers[:-1]:
    model.add(layer)

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [22]:
for layer in model.layers:
    layer.trainable=False

In [23]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [24]:
from keras.layers import Dense

In [25]:
model.add(Dense(1,activation='sigmoid'))

In [26]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [27]:
model.compile(optimizer='Adam',loss = 'binary_crossentropy',metrics=['accuracy'])


In [28]:
model.fit(X_train,Y_train,epochs=3,validation_data=(X_test,Y_test))

Epoch 1/3
41/41 [==============================] - 242s 6s/step - loss: 0.1175 - accuracy: 0.9510 - val_loss: 0.0244 - val_accuracy: 0.9878
Epoch 2/3
41/41 [==============================] - 243s 6s/step - loss: 0.0135 - accuracy: 0.9962 - val_loss: 0.0174 - val_accuracy: 0.9939
Epoch 3/3
41/41 [==============================] - 202s 5s/step - loss: 0.0086 - accuracy: 0.9992 - val_loss: 0.0173 - val_accuracy: 0.9908


In [72]:
def detect_face_mask(img):
    y_pred = model.predict(img.reshape(1,224,224,3))
    
    return y_pred

In [98]:
cap = cv2.VideoCapture(0)

haar = "haarcascade_frontalface_default.xml"
faceCascade = cv2.CascadeClassifier(haar)

while True:
    ret, frame = cap.read()
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = faceCascade.detectMultiScale(
        gray,
        scaleFactor=1.2,
        minNeighbors=5,
        minSize=(30, 30),
        #flags=cv2.cv.CV_HAAR_SCALE_IMAGE
    )


    # Draw a rectangle around the faces
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 3)
    
    img = cv2.resize(frame,(224,224))
    
    y_pred = detect_face_mask(img)
    
    if y_pred < 0.5:
        draw_label(frame,"with Mask",(30,30),(0,255,0))
    else:
        draw_label(frame,"Without mask",(30,30),(0,0,255))
        
    
    #print(y_pred)
    
    cv2.imshow('window',frame)
    
    if cv2.waitKey(1) & 0xFF == ord('x'):
        break

cap.release()
cv2.destroyAllWindows()

In [56]:
sample1 = cv2.imread('without_mask_3771.jpg')
sample1 = cv2.resize(sample1,(224,224))

In [57]:
detect_face_mask(sample1)

array([[0.9999962]], dtype=float32)

In [95]:
def draw_label(img,text,pos,bg_color):
    
    text_size = cv2.getTextSize(text,cv2.FONT_HERSHEY_SIMPLEX,1,3)
    
    end_x = pos[0] + text_size[0][0] + 2
    end_y = pos[1] + text_size[0][1] - 2
    
    cv2.rectangle(img,pos,(end_x,end_y),bg_color,cv2.FILLED)
    cv2.putText(img,text,pos,cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)
    